In [3]:
Pkg.update()

  Updating registry at `/opt/julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating git-repo `https://github.com/reworkhow/XSim.jl.git`
  Updating git-repo `https://github.com/reworkhow/JWAS.jl.git`
 Resolving package versions...
 Installed InstantiateFromURL ────────── v0.3.0
 Installed StatsPlots ────────────────── v0.11.0
 Installed PyPlot ────────────────────── v2.8.1
 Installed IJulia ────────────────────── v1.18.1
 Installed HDF5 ──────────────────────── v0.11.1
 Installed Plots ─────────────────────── v0.25.1
 Installed Distributions ─────────────── v0.20.0
 Installed CSV ───────────────────────── v0.5.5
 Installed IniFile ───────────────────── v0.5.0
 Installed AxisAlgorithms ────────────── v1.0.0
 Installed ProgressMeter ─────────────── v1.0.0
 Installed Parsers ───────────────────── v0.3.6
 Installed DataFrames ────────────────── v0.18.3
 Installed QuadGK ────────────────────── v2.0.4
 Installed Arpack ────────────────────── 

In [4]:
using Pkg
Pkg.add(PackageSpec(name="JWAS",rev="master"))

  Updating git-repo `https://github.com/reworkhow/JWAS.jl.git`
 Resolving package versions...
  Updating `/opt/julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `/opt/julia/environments/v1.1/Manifest.toml`
 [no changes]


In [20]:
#Pkg.develop(PackageSpec(path="/home/jovyan/rohan/Box Sync/JWAS.jl"))
#Pkg.free("JWAS")
#Pkg.add("StatsPlots")

In [1]:
using DataFrames              # package for working with data sets
using JWAS                    # package for Bayesian regression analyses, including BayesB and BayesCπ        
using JWAS:misc               # utility functions
using Distributions       
using Plots                   # package for plotting 
using LinearAlgebra,Statistics,Random,DelimitedFiles, DataFrames

┌ Info: Recompiling stale cache file /opt/julia/compiled/v1.1/Plots/ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1184


### Input marker and phenotype data

In [2]:
function readMatBin(fileName)
    genStr = open(fileName)
    n = read(genStr,Int64)
    p = read(genStr,Int64)
    M = zeros(n,p)
    for j in 1:p
        for i in 1:n
            M[i,j] = read(genStr,Float64)
        end
    end
    close(genStr)
    return M
end

function removeCols!(M,cols)
    return M[:, [!(i in cols) for i=1:size(M,2)]]
end

removeCols! (generic function with 1 method)

In [3]:
posQTL  = Int64.(vec(readdlm("posQTL.csv")))
beta    = readdlm("beta.csv")
M = readMatBin("genotypes.bin");

In [4]:
n,p = size(M)
simData  = readtable("phenotypes.csv",header=false,names=[:y])# reading in the simulated phenotypes into a data frame
phenData = DataFrame(id=1:n, y=simData[:y])
first(phenData,5)

┌ Warning: readtable is deprecated, use CSV.read from the CSV package instead
│   caller = ip:0x0
└ @ Core :-1


,id,y
,Int64,Float64⍰
1,1,-3.56863
2,2,1.73437
3,3,2.31795
4,4,-0.264018
5,5,-3.13096


In [5]:
phenTrain = phenData[1001:end,:]
first(phenTrain,5)

,id,y
,Int64,Float64⍰
1,1001,-6.22029
2,1002,-0.952557
3,1003,-9.66847
4,1004,-0.959437
5,1005,1.48486


In [6]:
resVar = var(simData[:y])/2
genVar = resVar

12.83523324152048

### Run BayesC$\pi$ using JWAS

In [7]:
ids = string.(1:size(M,1))                     # ids in genotype file are sequential numbers 1...n
model  = build_model("y = intercept",resVar)   # give model (except for marker part)
add_genotypes(model,M,genVar,header=false,rowID=ids,G_is_marker_variance=false);

21834 markers on 2000 individuals were added.


In [1]:
?runMCMC

search:

Couldn't find runMCMC
Perhaps you meant run


No documentation found.

Binding `runMCMC` does not exist.


In [9]:
MCMCFileNAME = "MCMCSamples"                  # place to put samples of marker effects
                                              # marker effect is set to zero if that locus is not in model
out=runMCMC(model, phenTrain,                 # tell JWAS to run analysis, for given model and data 
    Pi=0.0,                                   # intial value of π
    estimatePi=false,
    chain_length=6000,                        # length of chain
    printout_frequency=5000,                  # how often to show progress of analysis 
    printout_model_info=true,                 # tell JWAS to show the options used in this analysis
    methods="BayesC",                         # tell JWAS to run a BayesC analysis
    output_samples_frequency=20,              # how often to output sampled quantities
    output_samples_file=MCMCFileNAME,         # file name to output sampled marker effects
    output_PEV=true
);


The prior for marker effects variance is calculated from the genetic variance and π.
The mean of the prior for the marker effects variance is: 0.001683


A Linear Mixed Model was build using model equations:

y = intercept

Model Information:

Term            C/F          F/R            nLevels
intercept       factor       fixed                1

MCMC Information:

methods                                      BayesC
chain_length                                   6000
burnin                                            0
estimatePi                                    false
estimateScale                                 false
starting_value                                false
printout_frequency                             5000
output_samples_frequency                         20
constraint                                    false
missing_phenotypes                             true
update_priors_frequency                           0

Hyper-parameters Information:

residual variances:        

running MCMC for BayesC... 83%|████████████████████▊    |  ETA: 0:00:33


Posterior means at iteration: 5000
Residual variance: 12.97232
Marker effects variance: 0.001713


running MCMC for BayesC...100%|█████████████████████████| Time: 0:03:12


In [10]:
keys(out)

Base.KeySet for a Dict{Any,Any} with 5 entries. Keys:
  "Posterior mean of marker effects"
  "EBV_y"
  "Posterior mean of residual variance"
  "Posterior mean of marker effects variance"
  "Posterior mean of location parameters"

In [11]:
out["EBV_y"]

,ID,Estimate,PEV
,Any,Any,Float64
1,1,1.38171,5.60741
2,2,3.12125,4.49459
3,3,1.19827,5.56181
4,4,6.69629,4.56821
5,5,-1.48774,5.19194
6,6,-1.98634,5.52945
7,7,3.16881,4.86205
8,8,0.692546,5.10005
9,9,-0.289158,4.47047


In [12]:
res = GWAS("MCMCSamples_marker_effects_y.txt";header=true)

21834×2 Array{Any,2}:
 "1"      1.0
 "2"      1.0
 "3"      1.0
 "4"      1.0
 "5"      1.0
 "6"      1.0
 "7"      1.0
 "8"      1.0
 "9"      1.0
 "10"     1.0
 "11"     1.0
 "12"     1.0
 "13"     1.0
 ⋮           
 "21823"  1.0
 "21824"  1.0
 "21825"  1.0
 "21826"  1.0
 "21827"  1.0
 "21828"  1.0
 "21829"  1.0
 "21830"  1.0
 "21831"  1.0
 "21832"  1.0
 "21833"  1.0
 "21834"  1.0

In [15]:
[res[posQTL,:] beta  out["Posterior mean of marker effects"][posQTL,2]]

40×4 Array{Any,2}:
 "8729"   0.483333     1.13899     0.517909   
 "18201"  0.346667     0.742589    0.330605   
 "16771"  0.0         -0.416029    2.68797e-5 
 "14237"  0.0         -0.68567    -0.00337233 
 "11291"  0.0166667   -1.25694    -0.0253099  
 "15837"  0.0          0.060424    4.24308e-5 
 "17008"  0.0         -0.333503   -0.000654036
 "16115"  0.0          0.457435    0.000199527
 "1681"   0.0          0.0351858  -6.74777e-5 
 "15679"  0.663333     2.7329      1.53616    
 "2790"   0.0166667   -0.692791   -0.01069    
 "20843"  0.00333333  -0.411793   -0.00145473 
 "19915"  0.0         -0.146152   -0.000284847
 ⋮                                            
 "11391"  0.0         -0.132497   -0.000361095
 "17889"  0.04         0.426983    0.0192386  
 "7815"   0.266667    -1.64235    -0.37516    
 "18247"  0.123333    -1.10174    -0.140383   
 "21776"  0.0         -1.16958    -0.00232964 
 "1206"   0.00666667   0.15797     0.000745576
 "6447"   0.246667    -0.978558   -0.2606

In [95]:
#W,y,Va,Ve = M[1:1000,:],phenTrain[:y],0.07,13.3
function winTest(W,y,Va,Ve,wSize=100)
    W = W .- mean(W,dims=1)
    V = W*W'*Va + I*Ve
    #V = I*Ve
    Vi = inv(V)
    n,k = size(W)
    X = [ones(n) zeros(n)]  
    nWin = ceil(Int64,k/wSize)

    testStat = zeros(nWin)
    wStartV  = Array{Int64,1}(undef,nWin)
    wEndV    = Array{Int64,1}(undef,nWin)
    for i=1:nWin
        wStart = (i-1)*wSize + 1
        wEnd   = wStart + wSize - 1
        wEnd   = wEnd <= k ? wEnd : k
        X = [ones(n) W[:,collect(wStart:wEnd)]]
        lhsi   = inv(X'Vi*X + I*0.001)
        rhs = X'Vi*y
        betaHat = lhsi*rhs
        vBetaH = lhsi - I*Va
        testStat[i] = betaHat[2:end]'inv(vBetaH[2:end,2:end])*betaHat[2:end]
        wStartV[i] = wStart
        wEndV[i] = wEnd
    end
    return (wStartV,wEndV,testStat)
end 

winTest (generic function with 2 methods)

In [14]:
methods(GWAS)

# 3 methods for generic function "GWAS":
[1] GWAS(marker_effects_file; header) in JWAS.misc at /opt/julia/packages/JWAS/MHlxX/src/4.misc/src/GWAS.jl:7
[2] GWAS(marker_effect_file, mme; header, window_size, threshold, output_winVarProps) in JWAS.misc at /opt/julia/packages/JWAS/MHlxX/src/4.misc/src/GWAS.jl:35
[3] GWAS(marker_effects_file, map_file, mme; header, window_size, threshold, output_winVarProps) in JWAS.misc at /opt/julia/packages/JWAS/MHlxX/src/4.misc/src/GWAS.jl:105

In [15]:
winVar = GWAS("MCMCSamples_marker_effects_y.txt",model.output_genotypes;header=true,window_size=100,threshold=0.005,output_winVarProps=false)

Compute the posterior probability of association of the genomic window that explains more than 0.005 of the total genetic variance.


,wStart,wEnd,wSize,prGenVar,WPPA,PPA_t
,Int64,Int64,Int64,Float64,Float64,Float64
1,15601,15700,100,1.91,0.986667,0.986667
2,3101,3200,100,1.71,0.943333,0.965
3,8701,8800,100,0.97,0.813333,0.914444
4,19901,20000,100,1.07,0.76,0.875833
5,6401,6500,100,0.99,0.74,0.848667
6,3801,3900,100,0.78,0.736667,0.83
7,7801,7900,100,0.75,0.726667,0.815238
8,12901,13000,100,0.84,0.72,0.803333
9,15701,15800,100,0.74,0.713333,0.793333


In [102]:
sum(winVar[:prGenVar])

101.03999999999996

In [18]:
sortPosQTL = sort(posQTL);

In [19]:
PPA = 0.34
bigPPA = winVar[PPA .<= winVar[:WPPA],: ]

lowPos  = [findlast(sortPosQTL .<= row[1]) for row in eachrow(bigPPA)] 
highPos = [findfirst(sortPosQTL .>= row[2]) for row in eachrow(bigPPA)]   
wPos = [findfirst(bigPPA[i,1] .<= sortPosQTL .< bigPPA[i,2]) for i=1:size(bigPPA,1) ]

lowQTL  = [i == nothing ? 0 : sortPosQTL[i] for i in lowPos]
highQTL = [i == nothing ? 0 : sortPosQTL[i] for i in highPos]
wQTL    = [i == nothing ? 0 : sortPosQTL[i] for i in wPos]

res = DataFrame(
    wStart = bigPPA[:wStart],
    wEnd = bigPPA[:wEnd],
    wQTL = wQTL,
    oQTL = min.(bigPPA[:wStart]-lowQTL,highQTL-bigPPA[:wEnd]),
    prVar  = bigPPA[:prGenVar],
    WPPA   = bigPPA[:WPPA],
    PPA_t = bigPPA[:PPA_t]
    )

,wStart,wEnd,wQTL,oQTL,prVar,WPPA,PPA_t
,Int64,Int64,Int64,Int64,Float64,Float64,Float64
1,7801,7900,7815,96,5.7,1.0,1.0
2,15601,15700,15679,137,19.23,1.0,1.0
3,20201,20300,20210,252,6.43,1.0,1.0
4,20501,20600,20589,243,7.61,1.0,1.0
5,8701,8800,8729,228,4.45,0.996667,0.999333
6,6401,6500,6447,1205,3.88,0.993333,0.998333
7,3801,3900,3843,355,4.06,0.966667,0.99381
8,18201,18300,18201,0,5.98,0.966667,0.990417
9,3101,3200,3139,311,7.61,0.943333,0.985185


In [105]:
wSize = 100
winTest(M[1:1000,:],y[1:1000],Va,Ve,wSize)
srtIndx = sortperm(testStat,rev=true)
bigF = [wStartV[srtIndx] wEndV[srtIndx] testStat[srtIndx]]

219×3 Array{Float64,2}:
  1301.0   1400.0  221.426 
  7501.0   7600.0  219.96  
  4901.0   5000.0  204.866 
  6601.0   6700.0  202.336 
 12501.0  12600.0  200.426 
 20001.0  20100.0  198.554 
  2901.0   3000.0  192.879 
  8101.0   8200.0  192.754 
 12701.0  12800.0  192.553 
  4301.0   4400.0  189.845 
  6001.0   6100.0  187.563 
 12401.0  12500.0  185.368 
 18601.0  18700.0  184.861 
     ⋮                     
 10401.0  10500.0   94.3837
 11601.0  11700.0   92.8171
 12201.0  12300.0   91.6667
 18001.0  18100.0   88.549 
  1501.0   1600.0   88.084 
  1401.0   1500.0   87.3405
 14301.0  14400.0   85.5331
  2301.0   2400.0   85.4357
 14601.0  14700.0   85.4   
 13101.0  13200.0   83.5136
 14201.0  14300.0   47.6031
 21801.0  21834.0   25.4372

In [111]:
wPos = [findfirst(bigF[i,1] .<= sortPosQTL .< bigF[i,2]) for i=1:size(bigF,1) ]

219-element Array{Union{Nothing, Int64},1}:
   nothing
   nothing
 10       
   nothing
   nothing
   nothing
   nothing
   nothing
   nothing
  8       
   nothing
   nothing
   nothing
  ⋮       
   nothing
   nothing
   nothing
   nothing
   nothing
   nothing
   nothing
   nothing
   nothing
   nothing
 22       
   nothing

In [109]:
sortPosQTL

40-element Array{Int64,1}:
   285
  1206
  1681
  2790
  3139
  3843
  4255
  4349
  4454
  4953
  6447
  7705
  7815
     ⋮
 17008
 17889
 18201
 18247
 19677
 19915
 19949
 20210
 20589
 20843
 21226
 21776

In [20]:
(1 - 8/28)*100

71.42857142857143

In [21]:
(1 - 1/15)*100

93.33333333333333

In [88]:
Va,Ve = 0.7,13.3

(0.7, 13.3)

In [66]:
#function t_test(M,y,Va,Ve)
    W = W .- mean(W,dims=1)
    V = W*W'*Va + I*Ve
    V = I*Ve
    Vi = inv(V)

    n,k = size(W)
    X = [ones(n) zeros(n)]  

1000×2 Array{Float64,2}:
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 ⋮       
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0
 1.0  0.0

In [55]:
    tcalc = zeros(k)
    for i=1:1=k
        X[:,2] = W[:,i]
        lhsi   = inv(X'Vi*X)
        rhs = X'Vi*y
        betaHat = lhsi*rhs
        #tcalc[i] = betaHat[2]/sqrt(lhsi[2,2])
        tcalc[i] = betaHat[2]
    end 

In [56]:
pvals = ccdf.(Normal(0,1), tcalc)
pvals[pvals .< 0.05]

1-element Array{Float64,1}:
 0.026816102227243416

In [60]:
srtIndx = sortperm(vec(beta))
[tcalc[srtIndx] beta[srtIndx]]

40×2 Array{Float64,2}:
 -0.315159    -1.79167 
  0.0327568   -1.72478 
 -0.193849    -1.64235 
 -0.116496    -1.58839 
  0.0109232   -1.25694 
  0.398628    -1.16958 
  1.9298      -1.16384 
 -0.154986    -1.10174 
 -0.449875    -0.978558
 -0.361338    -0.727422
  0.111133    -0.692791
  0.553563    -0.68567 
  0.188779    -0.463916
  ⋮                    
  0.140443     0.426983
  0.00940731   0.457435
 -0.214029     0.578151
 -0.141314     0.738006
 -0.33507      0.742589
  0.0719255    0.755684
 -0.472743     0.882019
 -0.153497     0.898326
  0.361763     1.13899 
 -0.25496      1.73079 
  0.503377     1.8056  
 -0.267722     2.7329  

In [1]:
cor(beta,tcalc)

UndefVarError: UndefVarError: cor not defined